# Comptage des mots

In [1]:
from Pretraitement import Pretraitements
import pandas as pd
import numpy as np
base_path = "C:\\Users\\ACER\\Desktop\\UTC\\SY09\\PROJET\\sy09_russian-troll-tweets"

In [2]:
original_data=pd.read_csv(os.path.join(base_path, "Data\combined_csv.csv"))#, nrows=100000)

C:\Users\ACER\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,10,15,20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
account_categories = original_data["account_category"].unique()

In [4]:
account_categories

array(['RightTroll', 'NonEnglish', 'Fearmonger', 'LeftTroll', 'Unknown',
       'HashtagGamer', 'NewsFeed', 'Commercial'], dtype=object)

In [6]:
for category in account_categories:
    print(category)
    data=original_data[original_data['language']=='English'][original_data['account_category']==category]
    pt=Pretraitements(data)
    pt.print_most_freq_words(n=100)

RightTroll
<ipython-input-6-75c595efbe01>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data=original_data[original_data['language']=='English'][original_data['account_category']==category]


KeyboardInterrupt: 

# Création des plots

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [9]:
csv_files = os.listdir(os.path.join(base_path, "Data\most_common_words_categories"))

In [ ]:
for file in csv_files:
    df = pd.read_csv(os.path.join(base_path, f"Data\most_common_words_categories\{file}"), names=["word", "occurences"], header=None)[:10]
    print(df)
    # sns.barplot(data=df, y="word", x="occurences").set_title(f"{file.split('.')[0]} most common words")
    plt.bar(x=df["word"], height=df["occurences"])
    plt.xticks(rotation=70)
    plt.title(f"{file.split('.')[0]} most common words")
    plt.xlabel("Word")
    plt.ylabel("Occurences")
    plt.savefig(os.path.join(base_path, "Data\most_common_words_categories", f"{file.split('.')[0]}.svg"))
    plt.savefig(os.path.join(base_path, "Data\most_common_words_categories", f"{file.split('.')[0]}.png"))
    plt.show()


In [ ]:
df = pd.read_csv(os.path.join(base_path, f"Data\most_common_words_categories\{file}"), names=["word", "occurences"], header=None)[:10]


In [ ]:
df["occurences"] = df["occurences"] / df["occurences"].sum()

In [ ]:
for index, row in df.iterrows():
    print(row['word'], row['occurences'])

In [ ]:
float(df.loc[df['word'] == "just"]["occurences"].values) * 3

In [ ]:
corr_matrix = np.zeros((8, 8))

In [ ]:
for idxx, filex in enumerate(csv_files):
    print(idxx, filex.split(".")[0])
    dfx = pd.read_csv(os.path.join(base_path, f"Data\most_common_words_categories\{filex}"), names=["word", "occurences"], header=None)
    dfx["occurences"] = dfx["occurences"] / dfx["occurences"].sum()
    for idxy, filey in enumerate(csv_files):
        print("   ", idxy, filey.split(".")[0])
        val_sum = 0
        dfy = pd.read_csv(os.path.join(base_path, f"Data\most_common_words_categories\{filey}"), names=["word", "occurences"], header=None)
        dfy["occurences"] = dfy["occurences"] / dfy["occurences"].sum()
        for index, row in dfx.iterrows():
            val1 = row["occurences"]
            val2 = dfy.loc[dfy['word'] == row["word"]]["occurences"].values
            if val2.size != 0:
                # print(val2.size)
                val_sum += (val1 + val2) / 2
        corr_matrix[idxx][idxy] = val_sum

In [ ]:
corr_matrix

In [ ]:
axis_labels = ["Commercial","Fearmonger","HashtagGamer","LeftTroll","NewsFeed","NonEnglish","RightTroll","Unknown"] # labels for x-axis
ax = sns.heatmap(corr_matrix, xticklabels=axis_labels, yticklabels=axis_labels)
plt.savefig(os.path.join(base_path, "Data\most_common_words_categories", "words_in_common_count.svg"))
plt.savefig(os.path.join(base_path, "Data\most_common_words_categories", "words_in_common_count.png"))
plt.show()


# Ajout de variables associées au mots

In [10]:
EXP_REG_HASHTAG="\B#\w*\S+\w"
import re

def find_and_clean_words(text, keep_hashtags=False):
    """
    @return la liste des mots nettoyés d'un texte.
    @param keep_hashtags:
        - True: considère les hashtags comme des mots (les garde)
        - False: retire les hashtags.
    """
    text = text.lower()
    if keep_hashtags==False:
        text=re.sub(EXP_REG_HASHTAG,'',text)
    lst=re.findall("\w+",text)
    # for word in lst:
    #     word = word.lower()
    #     if len(word) > 4:
    #         res.append(word)
    return lst

def calculate_score_words(common_words, tweet_words):
    score = 0
    tweet_words = pd.DataFrame(tweet_words,columns=['tweet_words'])
    return pd.merge(common_words, tweet_words, how='inner', left_on='word', right_on='tweet_words')["occurences"].sum()

In [11]:
# generation du dataframe avec les colonnes de score concernant les mots contenus dans les tweets

new_data=original_data[original_data['language']=='English']
for idx, file in enumerate(csv_files):
    print(idx, file.split(".")[0])
    df = pd.read_csv(os.path.join(base_path, f"Data\most_common_words_categories\{file}"), names=["word", "occurences"], header=None)
    df["occurences"] = df["occurences"] / df["occurences"].sum()
    new_data[f"score_word_{file.split('.')[0]}"] = new_data.apply(lambda row : calculate_score_words(df, find_and_clean_words(row["content"])), axis = 1)

0 Commercial


AttributeError: 'float' object has no attribute 'lower'